In [ ]:
!pip install pycaret[full]

  Using cached matplotlib-3.7.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.7 kB)
  Using cached shap-0.44.1-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (24 kB)
  Using cached interpret-0.7.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached PyYAML-5.4.1.tar.gz (175 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [61 lines of output]
      /var/tmp/pbs.11346544.pbs101/pip-build-env-l2zyi0mu/overlay/lib/python3.10/site-packages/setuptools/dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider removing the following classifiers in favor of a SPDX license express

In [ ]:
!pip install skorch

In [ ]:
!pip install pytorch-ignite

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import tensor

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import RidgeClassifier, LogisticRegression, SGDClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from catboost import CatBoostClassifier

import shap
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, root_mean_squared_error, mean_squared_log_error, root_mean_squared_log_error, mean_absolute_percentage_error, median_absolute_error, max_error

import torch.nn as nn

from skorch import NeuralNetClassifier, NeuralNetRegressor
from sklearn.pipeline import Pipeline
from skorch.helper import DataFrameTransformer
import skorch

from sklearn.model_selection import RandomizedSearchCV

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
df = pd.read_csv('mech_prop_training_dataset.csv')
df

/var/tmp/pbs.11346544.pbs101/ipykernel_3130870/4254923679.py:1: DtypeWarning: Columns (31,43,65) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('For_mech_prop_predictions_13_July_2024.csv')


,material_id,K_VRH,G_VRH,Pugh_Ratio,builder_meta,nsites,elements,nelements,composition_reduced,formula_pretty,...,maximum local difference in GSmagmom,range local difference in GSmagmom,mean local difference in GSmagmom,avg_dev local difference in GSmagmom,minimum local difference in SpaceGroupNumber,maximum local difference in SpaceGroupNumber,range local difference in SpaceGroupNumber,mean local difference in SpaceGroupNumber,avg_dev local difference in SpaceGroupNumber,exists
0,mp-1,2.0,0.0,inf,emmet_version='0.72.20' pymatgen_version='2023...,1.0,[Element Cs],1.0,Cs1,Cs,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,True
1,mp-10,80.0,43.0,1.860465,emmet_version='0.72.20' pymatgen_version='2023...,2.0,[Element As],1.0,As1,As,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,True
2,mp-1000,26.0,16.0,1.625000,emmet_version='0.72.20' pymatgen_version='2023...,2.0,"[Element Ba, Element Te]",2.0,Ba1 Te1,BaTe,...,0.000000,0.000000e+00,0.000000,0.000000e+00,76.999987,76.999987,0.000000,76.999987,0.000000,True
3,mp-10000,131.0,74.0,1.770270,emmet_version='0.72.20' pymatgen_version='2023...,6.0,"[Element Hf, Element S]",2.0,Hf2 S1,Hf2S,...,0.000000,0.000000e+00,0.000000,0.000000e+00,44.415333,96.285363,51.870030,61.705343,23.053347,True
4,mp-10003,191.0,97.0,1.969072,emmet_version='0.72.20' pymatgen_version='2023...,12.0,"[Element Co, Element Nb, Element Si]",3.0,Nb4 Co1 Si1,Nb4CoSi,...,1.288343,1.288343e+00,0.396151,2.973975e-01,1.621693,29.120347,27.498654,9.467731,6.550872,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16498,mp-999550,132.0,48.0,2.750000,emmet_version='0.72.20' pymatgen_version='2023...,4.0,"[Element Ga, Element Mn]",2.0,Mn3 Ga1,Mn3Ga,...,0.000310,2.406508e-04,0.000155,7.753017e-05,34.273413,153.000000,118.726587,76.500000,38.250000,True
16499,mp-999558,205.0,89.0,2.303371,emmet_version='0.72.20' pymatgen_version='2023...,4.0,"[Element Mn, Element Ni, Element Si]",3.0,Mn2 Si1 Ni1,Mn2SiNi,...,0.595205,4.617801e-01,0.297697,1.487538e-01,5.344055,6.896037,1.551982,5.896037,0.500000,True
16500,mp-999576,232.0,115.0,2.017391,emmet_version='0.72.20' pymatgen_version='2023...,4.0,"[Element Mn, Element Ru, Element Si]",3.0,Mn2 Si1 Ru1,Mn2SiRu,...,0.000190,5.421011e-20,0.000190,2.710505e-20,9.032166,26.879954,17.847788,16.500000,6.401924,True
16501,mp-9996,12.0,10.0,1.200000,emmet_version='0.72.20' pymatgen_version='2023...,24.0,"[Element P, Element Si]",2.0,Si1 P2,SiP2,...,0.000000,0.000000e+00,0.000000,0.000000e+00,37.463038,186.001437,148.538399,112.163384,49.582687,True


In [ ]:
# compositional attributes only
compositional_features_only_df = df[['0-norm',
 '2-norm',
 '3-norm',
 '5-norm',
 '7-norm',
 '10-norm',
 'MagpieData minimum Number',
 'MagpieData maximum Number',
 'MagpieData range Number',
 'MagpieData mean Number',
 'MagpieData avg_dev Number',
 'MagpieData mode Number',
 'MagpieData minimum MendeleevNumber',
 'MagpieData maximum MendeleevNumber',
 'MagpieData range MendeleevNumber',
 'MagpieData mean MendeleevNumber',
 'MagpieData avg_dev MendeleevNumber',
 'MagpieData mode MendeleevNumber',
 'MagpieData minimum AtomicWeight',
 'MagpieData maximum AtomicWeight',
 'MagpieData range AtomicWeight',
 'MagpieData mean AtomicWeight',
 'MagpieData avg_dev AtomicWeight',
 'MagpieData mode AtomicWeight',
 'MagpieData minimum MeltingT',
 'MagpieData maximum MeltingT',
 'MagpieData range MeltingT',
 'MagpieData mean MeltingT',
 'MagpieData avg_dev MeltingT',
 'MagpieData mode MeltingT',
 'MagpieData minimum Column',
 'MagpieData maximum Column',
 'MagpieData range Column',
 'MagpieData mean Column',
 'MagpieData avg_dev Column',
 'MagpieData mode Column',
 'MagpieData minimum Row',
 'MagpieData maximum Row',
 'MagpieData range Row',
 'MagpieData mean Row',
 'MagpieData avg_dev Row',
 'MagpieData mode Row',
 'MagpieData minimum CovalentRadius',
 'MagpieData maximum CovalentRadius',
 'MagpieData range CovalentRadius',
 'MagpieData mean CovalentRadius',
 'MagpieData avg_dev CovalentRadius',
 'MagpieData mode CovalentRadius',
 'MagpieData minimum Electronegativity',
 'MagpieData maximum Electronegativity',
 'MagpieData range Electronegativity',
 'MagpieData mean Electronegativity',
 'MagpieData avg_dev Electronegativity',
 'MagpieData mode Electronegativity',
 'MagpieData minimum NsValence',
 'MagpieData maximum NsValence',
 'MagpieData range NsValence',
 'MagpieData mean NsValence',
 'MagpieData avg_dev NsValence',
 'MagpieData mode NsValence',
 'MagpieData minimum NpValence',
 'MagpieData maximum NpValence',
 'MagpieData range NpValence',
 'MagpieData mean NpValence',
 'MagpieData avg_dev NpValence',
 'MagpieData mode NpValence',
 'MagpieData minimum NdValence',
 'MagpieData maximum NdValence',
 'MagpieData range NdValence',
 'MagpieData mean NdValence',
 'MagpieData avg_dev NdValence',
 'MagpieData mode NdValence',
 'MagpieData minimum NfValence',
 'MagpieData maximum NfValence',
 'MagpieData range NfValence',
 'MagpieData mean NfValence',
 'MagpieData avg_dev NfValence',
 'MagpieData mode NfValence',
 'MagpieData minimum NValence',
 'MagpieData maximum NValence',
 'MagpieData range NValence',
 'MagpieData mean NValence',
 'MagpieData avg_dev NValence',
 'MagpieData mode NValence',
 'MagpieData minimum NsUnfilled',
 'MagpieData maximum NsUnfilled',
 'MagpieData range NsUnfilled',
 'MagpieData mean NsUnfilled',
 'MagpieData avg_dev NsUnfilled',
 'MagpieData mode NsUnfilled',
 'MagpieData minimum NpUnfilled',
 'MagpieData maximum NpUnfilled',
 'MagpieData range NpUnfilled',
 'MagpieData mean NpUnfilled',
 'MagpieData avg_dev NpUnfilled',
 'MagpieData mode NpUnfilled',
 'MagpieData minimum NdUnfilled',
 'MagpieData maximum NdUnfilled',
 'MagpieData range NdUnfilled',
 'MagpieData mean NdUnfilled',
 'MagpieData avg_dev NdUnfilled',
 'MagpieData mode NdUnfilled',
 'MagpieData minimum NfUnfilled',
 'MagpieData maximum NfUnfilled',
 'MagpieData range NfUnfilled',
 'MagpieData mean NfUnfilled',
 'MagpieData avg_dev NfUnfilled',
 'MagpieData mode NfUnfilled',
 'MagpieData minimum NUnfilled',
 'MagpieData maximum NUnfilled',
 'MagpieData range NUnfilled',
 'MagpieData mean NUnfilled',
 'MagpieData avg_dev NUnfilled',
 'MagpieData mode NUnfilled',
 'MagpieData minimum GSvolume_pa',
 'MagpieData maximum GSvolume_pa',
 'MagpieData range GSvolume_pa',
 'MagpieData mean GSvolume_pa',
 'MagpieData avg_dev GSvolume_pa',
 'MagpieData mode GSvolume_pa',
 'MagpieData minimum GSbandgap',
 'MagpieData maximum GSbandgap',
 'MagpieData range GSbandgap',
 'MagpieData mean GSbandgap',
 'MagpieData avg_dev GSbandgap',
 'MagpieData mode GSbandgap',
 'MagpieData minimum GSmagmom',
 'MagpieData maximum GSmagmom',
 'MagpieData range GSmagmom',
 'MagpieData mean GSmagmom',
 'MagpieData avg_dev GSmagmom',
 'MagpieData mode GSmagmom',
 'MagpieData minimum SpaceGroupNumber',
 'MagpieData maximum SpaceGroupNumber',
 'MagpieData range SpaceGroupNumber',
 'MagpieData mean SpaceGroupNumber',
 'MagpieData avg_dev SpaceGroupNumber',
 'MagpieData mode SpaceGroupNumber',
 'frac s valence electrons',
 'frac p valence electrons',
 'frac d valence electrons',
 'frac f valence electrons',
 'compound possible',
 'max ionic char',
 'avg ionic char', 'K_VRH', 'G_VRH', 'Pugh_Ratio']]
compositional_features_only_df

,0-norm,2-norm,3-norm,5-norm,7-norm,10-norm,MagpieData minimum Number,MagpieData maximum Number,MagpieData range Number,MagpieData mean Number,...,frac s valence electrons,frac p valence electrons,frac d valence electrons,frac f valence electrons,compound possible,max ionic char,avg ionic char,K_VRH,G_VRH,Pugh_Ratio
0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,55.0,55.0,0.0,55.000000,...,1.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,2.0,0.0,inf
1,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,33.0,33.0,0.0,33.000000,...,0.133333,0.200000,0.666667,0.000000,1.0,0.000000,0.000000,80.0,43.0,1.860465
2,2.0,0.707107,0.629961,0.574349,0.552045,0.535887,52.0,56.0,4.0,54.000000,...,0.222222,0.222222,0.555556,0.000000,1.0,0.306515,0.076629,26.0,16.0,1.625000
3,2.0,0.745356,0.693361,0.670782,0.667408,0.666732,16.0,72.0,56.0,53.333333,...,0.142857,0.095238,0.095238,0.666667,0.0,0.336084,0.074685,131.0,74.0,1.770270
4,3.0,0.707107,0.673540,0.666927,0.666678,0.666667,14.0,41.0,27.0,34.166667,...,0.242424,0.060606,0.696970,0.000000,0.0,0.022249,0.004631,191.0,97.0,1.969072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16498,2.0,0.790569,0.759147,0.750616,0.750049,0.750001,25.0,31.0,6.0,26.500000,...,0.235294,0.029412,0.735294,0.000000,0.0,0.016758,0.003142,132.0,48.0,2.750000
16499,3.0,0.612372,0.538609,0.506099,0.501109,0.500098,14.0,28.0,14.0,23.000000,...,0.285714,0.071429,0.642857,0.000000,0.0,0.031881,0.007757,205.0,89.0,2.303371
16500,3.0,0.612372,0.538609,0.506099,0.501109,0.500098,14.0,44.0,30.0,27.000000,...,0.269231,0.076923,0.653846,0.000000,0.0,0.100238,0.017690,232.0,115.0,2.017391
16501,2.0,0.745356,0.693361,0.670782,0.667408,0.666732,14.0,15.0,1.0,14.666667,...,0.428571,0.571429,0.000000,0.000000,0.0,0.020806,0.004623,12.0,10.0,1.200000


In [ ]:
X_comp_features = compositional_features_only_df.drop(['K_VRH', 'G_VRH', 'Pugh_Ratio'], axis = 1)
y_comp_features = compositional_features_only_df[['K_VRH', 'G_VRH',	'Pugh_Ratio']]

In [ ]:
compositional_features_only_df[compositional_features_only_df.columns[0:100]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16503 entries, 0 to 16502
Data columns (total 100 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   0-norm                                16503 non-null  float64
 1   2-norm                                16503 non-null  float64
 2   3-norm                                16503 non-null  float64
 3   5-norm                                16503 non-null  float64
 4   7-norm                                16503 non-null  float64
 5   10-norm                               16503 non-null  float64
 6   MagpieData minimum Number             16503 non-null  float64
 7   MagpieData maximum Number             16503 non-null  float64
 8   MagpieData range Number               16503 non-null  float64
 9   MagpieData mean Number                16503 non-null  float64
 10  MagpieData avg_dev Number             16503 non-null  float64
 11  MagpieData mod

In [ ]:
compositional_features_only_df[compositional_features_only_df.columns[100:]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16503 entries, 0 to 16502
Data columns (total 48 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   MagpieData avg_dev NdUnfilled        16503 non-null  float64
 1   MagpieData mode NdUnfilled           16503 non-null  float64
 2   MagpieData minimum NfUnfilled        16503 non-null  float64
 3   MagpieData maximum NfUnfilled        16503 non-null  float64
 4   MagpieData range NfUnfilled          16503 non-null  float64
 5   MagpieData mean NfUnfilled           16503 non-null  float64
 6   MagpieData avg_dev NfUnfilled        16503 non-null  float64
 7   MagpieData mode NfUnfilled           16503 non-null  float64
 8   MagpieData minimum NUnfilled         16503 non-null  float64
 9   MagpieData maximum NUnfilled         16503 non-null  float64
 10  MagpieData range NUnfilled           16503 non-null  float64
 11  MagpieData mean NUnfilled   

In [ ]:
df.columns[0:100]

Index(['material_id', 'K_VRH', 'G_VRH', 'Pugh_Ratio', 'builder_meta', 'nsites',
       'elements', 'nelements', 'composition_reduced', 'formula_pretty',
       'formula_anonymous', 'chemsys', 'volume', 'density', 'density_atomic',
       'symmetry', 'property_name', 'deprecated', 'deprecation_reasons',
       'last_updated', 'origins', 'warnings', 'task_ids',
       'uncorrected_energy_per_atom', 'energy_per_atom',
       'formation_energy_per_atom', 'energy_above_hull', 'is_stable',
       'equilibrium_reaction_energy_per_atom', 'decomposes_to', 'xas',
       'grain_boundaries', 'band_gap', 'cbm', 'vbm', 'efermi', 'is_gap_direct',
       'is_metal', 'es_source_calc_id', 'bandstructure', 'dos',
       'dos_energy_up', 'dos_energy_down', 'is_magnetic', 'ordering',
       'total_magnetization', 'total_magnetization_normalized_vol',
       'total_magnetization_normalized_formula_units', 'num_magnetic_sites',
       'num_unique_magnetic_sites', 'types_of_magnetic_species',
       'bulk_mod

In [ ]:
df.columns[100:]

Index(['Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As',
       ...
       'maximum local difference in GSmagmom',
       'range local difference in GSmagmom',
       'mean local difference in GSmagmom',
       'avg_dev local difference in GSmagmom',
       'minimum local difference in SpaceGroupNumber',
       'maximum local difference in SpaceGroupNumber',
       'range local difference in SpaceGroupNumber',
       'mean local difference in SpaceGroupNumber',
       'avg_dev local difference in SpaceGroupNumber', 'exists'],
      dtype='object', length=361)

In [ ]:
import torch.nn as nn

from skorch import NeuralNetClassifier, NeuralNetRegressor
from sklearn.pipeline import Pipeline
from skorch.helper import DataFrameTransformer

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16503 entries, 0 to 16502
Columns: 461 entries, material_id to exists
dtypes: bool(7), float64(424), object(30)
memory usage: 57.3+ MB


In [ ]:
print("There are {} columns of names listed below:\n\n{}".format(df.shape[1], df.columns.values))

There are 461 columns of names listed below:

['material_id' 'K_VRH' 'G_VRH' 'Pugh_Ratio' 'builder_meta' 'nsites'
 'elements' 'nelements' 'composition_reduced' 'formula_pretty'
 'formula_anonymous' 'chemsys' 'volume' 'density' 'density_atomic'
 'symmetry' 'property_name' 'deprecated' 'deprecation_reasons'
 'last_updated' 'origins' 'warnings' 'task_ids'
 'uncorrected_energy_per_atom' 'energy_per_atom'
 'formation_energy_per_atom' 'energy_above_hull' 'is_stable'
 'equilibrium_reaction_energy_per_atom' 'decomposes_to' 'xas'
 'grain_boundaries' 'band_gap' 'cbm' 'vbm' 'efermi' 'is_gap_direct'
 'is_metal' 'es_source_calc_id' 'bandstructure' 'dos' 'dos_energy_up'
 'dos_energy_down' 'is_magnetic' 'ordering' 'total_magnetization'
 'total_magnetization_normalized_vol'
 'total_magnetization_normalized_formula_units' 'num_magnetic_sites'
 'num_unique_magnetic_sites' 'types_of_magnetic_species' 'bulk_modulus'
 'shear_modulus' 'universal_anisotropy' 'homogeneous_poisson' 'e_total'
 'e_ionic' 'e_elec

In [ ]:
df3 = pd.read_csv('mech_prop_validation_dataset.csv')
df3

,material_id,builder_meta,nsites,elements,nelements,composition_reduced,formula_pretty,formula_anonymous,chemsys,volume,...,avg_dev local difference in GSmagmom,minimum local difference in SpaceGroupNumber,maximum local difference in SpaceGroupNumber,range local difference in SpaceGroupNumber,mean local difference in SpaceGroupNumber,avg_dev local difference in SpaceGroupNumber,K_VRH,G_VRH,Pugh_Ratio,exists
0,mp-3949,emmet_version='0.72.20' pymatgen_version='2023...,64,"[Element K, Element Li, Element Si]",3,K7 Li1 Si8,K7LiSi8,AB7C8,K-Li-Si,1951.965448,...,0.0,1.049271,1.424875,0.375604,1.266754,0.135871,NaN,NaN,NaN,False
1,mp-4657,emmet_version='0.72.20' pymatgen_version='2023...,80,"[Element Ga, Element Li, Element O]",3,Li5 Ga1 O4,Li5GaO4,AB4C5,Ga-Li-O,748.454369,...,0.0,86.530203,191.228152,104.697949,151.220931,27.036697,NaN,NaN,NaN,False
2,mp-4995,emmet_version='0.72.20' pymatgen_version='2023...,20,"[Element Li, Element O, Element Sb]",3,Li1 Sb1 O3,LiSbO3,ABC3,Li-O-Sb,215.500133,...,0.0,100.642918,205.344802,104.701884,134.920881,35.801216,NaN,NaN,NaN,False
3,mp-5368,emmet_version='0.72.20' pymatgen_version='2023...,27,"[Element F, Element Ge, Element Li]",3,Li2 Ge1 F6,Li2GeF6,AB2C6,F-Ge-Li,272.775032,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,mp-5488,emmet_version='0.72.20' pymatgen_version='2023...,8,"[Element In, Element Li, Element O]",3,Li1 In1 O2,LiInO2,ABC2,In-Li-O,87.122210,...,0.0,126.862161,204.842529,77.980367,166.352988,19.745413,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4738,mp-2712854,emmet_version='0.72.20' pymatgen_version='2023...,102,"[Element Al, Element Ge, Element O, Element P,...",5,Al1 Ge1 P1 S2 O12,AlGeP(SO6)2,ABCD2E12,Al-Ge-O-P-S,1283.361072,...,0.0,10.000000,213.000000,203.000000,52.359925,39.124741,NaN,NaN,NaN,False
4739,mp-2712928,emmet_version='0.72.20' pymatgen_version='2023...,102,"[Element Al, Element O, Element P, Element S, ...",5,Al1 Sn1 P1 S2 O12,AlSnP(SO6)2,ABCD2E12,Al-O-P-S-Sn,1394.547188,...,0.0,10.000000,213.000000,203.000000,44.558663,32.913571,NaN,NaN,NaN,False
4740,mp-2713730,emmet_version='0.72.20' pymatgen_version='2023...,102,"[Element Al, Element Ga, Element O, Element S]",4,Al1 Ga1 S3 O12,AlGa(SO4)3,ABC3D12,Al-Ga-O-S,1307.718619,...,0.0,13.475931,213.000000,199.524069,41.671618,27.134342,NaN,NaN,NaN,False
4741,mp-2713752,emmet_version='0.72.20' pymatgen_version='2023...,102,"[Element Al, Element O, Element S]",3,Al2 S3 O12,Al2(SO4)3,A2B3C12,Al-O-S,1272.270712,...,0.0,19.363213,213.000000,193.636787,56.762943,37.198269,NaN,NaN,NaN,False


In [ ]:
len(X_comp_features.columns)

145

In [ ]:
torch.manual_seed(0)

In [ ]:
from torch import nn
import torch.nn.functional as F

class Regressor(nn.Module):
    def __init__(self, num_inputs=len(X_comp_features.columns), num_units_d1=300, num_units_d2=300, dropout = 0.2):
        super(Regressor, self).__init__()

        self.first_layer = nn.Linear(num_inputs, num_units_d1)
        self.batch_norm1 = nn.BatchNorm1d(num_units_d1)
        self.dropout = nn.Dropout(dropout)
        self.second_layer = nn.Linear(num_units_d1,num_units_d2)
        self.batch_norm2 = nn.BatchNorm1d(num_units_d2)
        self.final_layer = nn.Linear(num_units_d2,1)

    def forward(self, x_batch):
        X = self.first_layer(x_batch)
        X = self.batch_norm1(X)
        X = F.relu(X)
        X= self.dropout(X)
        X = self.second_layer(X)
        X = self.batch_norm2(X)
        X = F.relu(X)

        return self.final_layer(X)

In [ ]:
from skorch import NeuralNetRegressor
from torch import optim

skorch_regressor = NeuralNetRegressor(module=Regressor, optimizer=optim.Adam, max_epochs=200, verbose=0, lr = 0.0001, batch_size = 128)

skorch_regressor

<class 'skorch.regressor.NeuralNetRegressor'>[uninitialized](
  module=<class '__main__.Regressor'>,
)

In [ ]:
skorch_regressor.initialize().load_params(
                              f_params="params.pkl",
                              f_optimizer="opt.pkl",
                              f_criterion="criterion.pkl",
                              f_history="hist.json"
                            )

In [ ]:
X_train, X_holdout, y_train, y_holdout = train_test_split(X_comp_features, y_comp_features['K_VRH'], train_size=0.8, random_state=0)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_holdout = scaler.transform(X_holdout)

In [ ]:
from torch import tensor
X_train = tensor(X_train, dtype=torch.float32)
X_holdout = tensor(X_holdout, dtype=torch.float32)
y_train = tensor(y_train.values.reshape(-1,1), dtype=torch.float32)
y_holdout = tensor(y_holdout.values.reshape(-1,1), dtype=torch.float32)

X_train.shape, X_holdout.shape, y_train.shape, y_holdout.shape

(torch.Size([13202, 145]),
 torch.Size([3301, 145]),
 torch.Size([13202, 1]),
 torch.Size([3301, 1]))

In [ ]:
skorch_regressor.fit(X_train, y_train)

<class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=Regressor(
    (first_layer): Linear(in_features=145, out_features=300, bias=True)
    (batch_norm1): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (dropout): Dropout(p=0.2, inplace=False)
    (second_layer): Linear(in_features=300, out_features=300, bias=True)
    (batch_norm2): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (final_layer): Linear(in_features=300, out_features=1, bias=True)
  ),
)

In [ ]:
y_holdout_preds = skorch_regressor.predict(X_holdout)

In [ ]:
type(y_holdout_preds)

numpy.ndarray

In [ ]:
y_holdout_preds.shape

(3301, 1)

In [ ]:
print("Train MSE : {}".format(mean_squared_error(y_train, skorch_regressor.predict(X_train).reshape(-1))))
print("Holdout  MSE : {}".format(mean_squared_error(y_holdout, skorch_regressor.predict(X_holdout).reshape(-1))))

print("\nTrain R^2 : {}".format(skorch_regressor.score(X_train, y_train)))
print("Holdout  R^2 : {}".format(skorch_regressor.score(X_holdout, y_holdout)))

Train MSE : 566.4871826171875
Holdout  MSE : 717.8636474609375

Train R^2 : 0.8719737109582304
Holdout  R^2 : 0.8320067417509485


In [ ]:
skorch_regressor.get_params()

{'module': __main__.Regressor,
 'criterion': torch.nn.modules.loss.MSELoss,
 'optimizer': torch.optim.adam.Adam,
 'lr': 0.0001,
 'max_epochs': 200,
 'batch_size': 128,
 'iterator_train': torch.utils.data.dataloader.DataLoader,
 'iterator_valid': torch.utils.data.dataloader.DataLoader,
 'dataset': skorch.dataset.Dataset,
 'train_split': <skorch.dataset.ValidSplit object at 0x15181cabd330>,
 'callbacks': None,
 'predict_nonlinearity': 'auto',
 'warm_start': False,
 'verbose': 0,
 'device': 'cpu',
 'compile': False,
 'use_caching': 'auto',
 '_params_to_validate': set(),
 'callbacks__epoch_timer': <skorch.callbacks.logging.EpochTimer at 0x151815e630d0>,
 'callbacks__train_loss': <skorch.callbacks.scoring.PassthroughScoring at 0x151815e62dd0>,
 'callbacks__train_loss__name': 'train_loss',
 'callbacks__train_loss__lower_is_better': True,
 'callbacks__train_loss__on_train': True,
 'callbacks__valid_loss': <skorch.callbacks.scoring.PassthroughScoring at 0x151815e63610>,
 'callbacks__valid_loss

In [ ]:
skorch_regressor

<class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=Regressor(
    (first_layer): Linear(in_features=145, out_features=300, bias=True)
    (batch_norm1): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (dropout): Dropout(p=0.2, inplace=False)
    (second_layer): Linear(in_features=300, out_features=300, bias=True)
    (batch_norm2): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (final_layer): Linear(in_features=300, out_features=1, bias=True)
  ),
)

In [ ]:
TRAIN_PRED = pd.DataFrame()
HOLDOUT_PRED = pd.DataFrame()

NEW_TEST_PRED_2 = pd.DataFrame()
NEW_TEST_PROBA_2 = pd.DataFrame()

TRAIN_R2 = pd.DataFrame()
TRAIN_MAE = pd.DataFrame()
TRAIN_MSE = pd.DataFrame()
TRAIN_RMSE = pd.DataFrame()
TRAIN_MSLE = pd.DataFrame()
TRAIN_RMSLE = pd.DataFrame()
TRAIN_MAPE = pd.DataFrame()
TRAIN_MEDAE = pd.DataFrame()
TRAIN_MAXE = pd.DataFrame()

HOLDOUT_R2 = pd.DataFrame()
HOLDOUT_MAE = pd.DataFrame()
HOLDOUT_MSE = pd.DataFrame()
HOLDOUT_RMSE = pd.DataFrame()
HOLDOUT_MSLE = pd.DataFrame()
HOLDOUT_RMSLE = pd.DataFrame()
HOLDOUT_MAPE = pd.DataFrame()
HOLDOUT_MEDAE = pd.DataFrame()
HOLDOUT_MAXE = pd.DataFrame()

In [ ]:
models= [skorch_regressor, skorch_regressor, skorch_regressor]

models_name = ['skorch_model_1', 'skorch_model_2', 'skorch_model_3']

In [ ]:
import time

In [ ]:
from sklearn.metrics import PredictionErrorDisplay

for reg_n, reg in zip(models_name,models):

    TRAIN_R2_result = []
    TRAIN_MAE_result = []
    TRAIN_MSE_result = []
    TRAIN_RMSE_result = []

    TRAIN_MAPE_result = []
    TRAIN_MEDAE_result = []
    TRAIN_MAXE_result = []

    HOLDOUT_R2_result = []
    HOLDOUT_MAE_result = []
    HOLDOUT_MSE_result = []
    HOLDOUT_RMSE_result = []

    HOLDOUT_MAPE_result = []
    HOLDOUT_MEDAE_result = []
    HOLDOUT_MAXE_result = []


    for i in range(0,100):
        print(time.ctime())
        # Train-test split of 80:20 ratio to produce train and holdout data
        X_train, X_holdout, y_train, y_holdout = train_test_split(X_comp_features, y_comp_features['K_VRH'], train_size=0.8, random_state=0)

        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_holdout = scaler.transform(X_holdout)

        X_train = tensor(X_train, dtype=torch.float32)
        X_holdout = tensor(X_holdout, dtype=torch.float32)
        y_train = tensor(y_train.values.reshape(-1,1), dtype=torch.float32)
        y_holdout = tensor(y_holdout.values.reshape(-1,1), dtype=torch.float32)

        # compositional attributes only
        NEW_TEST_DB2 = df3[['0-norm',
                          '2-norm',
                          '3-norm',
                          '5-norm',
                          '7-norm',
                          '10-norm',
                          'MagpieData minimum Number',
                          'MagpieData maximum Number',
                          'MagpieData range Number',
                          'MagpieData mean Number',
                          'MagpieData avg_dev Number',
                          'MagpieData mode Number',
                          'MagpieData minimum MendeleevNumber',
                          'MagpieData maximum MendeleevNumber',
                          'MagpieData range MendeleevNumber',
                          'MagpieData mean MendeleevNumber',
                          'MagpieData avg_dev MendeleevNumber',
                          'MagpieData mode MendeleevNumber',
                          'MagpieData minimum AtomicWeight',
                          'MagpieData maximum AtomicWeight',
                          'MagpieData range AtomicWeight',
                          'MagpieData mean AtomicWeight',
                          'MagpieData avg_dev AtomicWeight',
                          'MagpieData mode AtomicWeight',
                          'MagpieData minimum MeltingT',
                          'MagpieData maximum MeltingT',
                          'MagpieData range MeltingT',
                          'MagpieData mean MeltingT',
                          'MagpieData avg_dev MeltingT',
                          'MagpieData mode MeltingT',
                          'MagpieData minimum Column',
                          'MagpieData maximum Column',
                          'MagpieData range Column',
                          'MagpieData mean Column',
                          'MagpieData avg_dev Column',
                          'MagpieData mode Column',
                          'MagpieData minimum Row',
                          'MagpieData maximum Row',
                          'MagpieData range Row',
                          'MagpieData mean Row',
                          'MagpieData avg_dev Row',
                          'MagpieData mode Row',
                          'MagpieData minimum CovalentRadius',
                          'MagpieData maximum CovalentRadius',
                          'MagpieData range CovalentRadius',
                          'MagpieData mean CovalentRadius',
                          'MagpieData avg_dev CovalentRadius',
                          'MagpieData mode CovalentRadius',
                          'MagpieData minimum Electronegativity',
                          'MagpieData maximum Electronegativity',
                          'MagpieData range Electronegativity',
                          'MagpieData mean Electronegativity',
                          'MagpieData avg_dev Electronegativity',
                          'MagpieData mode Electronegativity',
                          'MagpieData minimum NsValence',
                          'MagpieData maximum NsValence',
                          'MagpieData range NsValence',
                          'MagpieData mean NsValence',
                          'MagpieData avg_dev NsValence',
                          'MagpieData mode NsValence',
                          'MagpieData minimum NpValence',
                          'MagpieData maximum NpValence',
                          'MagpieData range NpValence',
                          'MagpieData mean NpValence',
                          'MagpieData avg_dev NpValence',
                          'MagpieData mode NpValence',
                          'MagpieData minimum NdValence',
                          'MagpieData maximum NdValence',
                          'MagpieData range NdValence',
                          'MagpieData mean NdValence',
                          'MagpieData avg_dev NdValence',
                          'MagpieData mode NdValence',
                          'MagpieData minimum NfValence',
                          'MagpieData maximum NfValence',
                          'MagpieData range NfValence',
                          'MagpieData mean NfValence',
                          'MagpieData avg_dev NfValence',
                          'MagpieData mode NfValence',
                          'MagpieData minimum NValence',
                          'MagpieData maximum NValence',
                          'MagpieData range NValence',
                          'MagpieData mean NValence',
                          'MagpieData avg_dev NValence',
                          'MagpieData mode NValence',
                          'MagpieData minimum NsUnfilled',
                          'MagpieData maximum NsUnfilled',
                          'MagpieData range NsUnfilled',
                          'MagpieData mean NsUnfilled',
                          'MagpieData avg_dev NsUnfilled',
                          'MagpieData mode NsUnfilled',
                          'MagpieData minimum NpUnfilled',
                          'MagpieData maximum NpUnfilled',
                          'MagpieData range NpUnfilled',
                          'MagpieData mean NpUnfilled',
                          'MagpieData avg_dev NpUnfilled',
                          'MagpieData mode NpUnfilled',
                          'MagpieData minimum NdUnfilled',
                          'MagpieData maximum NdUnfilled',
                          'MagpieData range NdUnfilled',
                          'MagpieData mean NdUnfilled',
                          'MagpieData avg_dev NdUnfilled',
                          'MagpieData mode NdUnfilled',
                          'MagpieData minimum NfUnfilled',
                          'MagpieData maximum NfUnfilled',
                          'MagpieData range NfUnfilled',
                          'MagpieData mean NfUnfilled',
                          'MagpieData avg_dev NfUnfilled',
                          'MagpieData mode NfUnfilled',
                          'MagpieData minimum NUnfilled',
                          'MagpieData maximum NUnfilled',
                          'MagpieData range NUnfilled',
                          'MagpieData mean NUnfilled',
                          'MagpieData avg_dev NUnfilled',
                          'MagpieData mode NUnfilled',
                          'MagpieData minimum GSvolume_pa',
                          'MagpieData maximum GSvolume_pa',
                          'MagpieData range GSvolume_pa',
                          'MagpieData mean GSvolume_pa',
                          'MagpieData avg_dev GSvolume_pa',
                          'MagpieData mode GSvolume_pa',
                          'MagpieData minimum GSbandgap',
                          'MagpieData maximum GSbandgap',
                          'MagpieData range GSbandgap',
                          'MagpieData mean GSbandgap',
                          'MagpieData avg_dev GSbandgap',
                          'MagpieData mode GSbandgap',
                          'MagpieData minimum GSmagmom',
                          'MagpieData maximum GSmagmom',
                          'MagpieData range GSmagmom',
                          'MagpieData mean GSmagmom',
                          'MagpieData avg_dev GSmagmom',
                          'MagpieData mode GSmagmom',
                          'MagpieData minimum SpaceGroupNumber',
                          'MagpieData maximum SpaceGroupNumber',
                          'MagpieData range SpaceGroupNumber',
                          'MagpieData mean SpaceGroupNumber',
                          'MagpieData avg_dev SpaceGroupNumber',
                          'MagpieData mode SpaceGroupNumber',
                          'frac s valence electrons',
                          'frac p valence electrons',
                          'frac d valence electrons',
                          'frac f valence electrons',
                          'compound possible',
                          'max ionic char',
                          'avg ionic char']]
        NEW_TEST_DB2 = scaler.transform(NEW_TEST_DB2)
        NEW_TEST_DB2 = tensor(NEW_TEST_DB2, dtype=torch.float32)


        # Train the model and perform predictions
        reg.fit(X_train,y_train)
        y_train_pred = reg.predict(X_train)
        y_holdout_pred = reg.predict(X_holdout)

        X_train = pd.DataFrame(X_train.numpy())
        X_holdout = pd.DataFrame(X_holdout.numpy())
        y_train = pd.DataFrame(y_train.numpy())
        y_holdout = pd.DataFrame(y_holdout.numpy())

        new_test_pred_2 = reg.predict(NEW_TEST_DB2)

        # Append to dataframe the prediced target labels (Train, holdout, and test) made by the models
        Train_Pred = pd.DataFrame(y_train_pred, columns=[str(reg_n)+'_'+str(i)])
        Holdout_Pred = pd.DataFrame(y_holdout_pred, columns=[str(reg_n)+'_'+str(i)])
        New_Test_pred_2 = pd.DataFrame(new_test_pred_2, columns=[str(reg_n)+'_'+str(i)])

        # Training performance metrics scores
        Train_r2 = r2_score(y_train, y_train_pred)
        Train_mae = mean_absolute_error(y_train, y_train_pred)
        Train_mse = mean_squared_error(y_train, y_train_pred)
        Train_rmse = root_mean_squared_error(y_train, y_train_pred)

        Train_mape = mean_absolute_percentage_error(y_train, y_train_pred)
        Train_medae = median_absolute_error(y_train, y_train_pred)
        Train_maxe = max_error(y_train, y_train_pred)

        # Holdout performance metrics scores
        Holdout_r2 = r2_score(y_holdout, y_holdout_pred)
        Holdout_mae = mean_absolute_error(y_holdout, y_holdout_pred)
        Holdout_mse = mean_squared_error(y_holdout, y_holdout_pred)
        Holdout_rmse = root_mean_squared_error(y_holdout, y_holdout_pred)

        Holdout_mape = mean_absolute_percentage_error(y_holdout, y_holdout_pred)
        Holdout_medae = median_absolute_error(y_holdout, y_holdout_pred)
        Holdout_maxe = max_error(y_holdout, y_holdout_pred)

        # Append training performance metrics scores of each model to dataframes
        TRAIN_R2_result.append(Train_r2)
        TRAIN_MAE_result.append(Train_mae)
        TRAIN_MSE_result.append(Train_mse)
        TRAIN_RMSE_result.append(Train_rmse)
        TRAIN_MAPE_result.append(Train_mape)
        TRAIN_MEDAE_result.append(Train_medae)
        TRAIN_MAXE_result.append(Train_maxe)

        # Append holdout performance metrics scores of each model to dataframes
        HOLDOUT_R2_result.append(Holdout_r2)
        HOLDOUT_MAE_result.append(Holdout_mae)
        HOLDOUT_MSE_result.append(Holdout_mse)
        HOLDOUT_RMSE_result.append(Holdout_rmse)
        HOLDOUT_MAPE_result.append(Holdout_mape)
        HOLDOUT_MEDAE_result.append(Holdout_medae)
        HOLDOUT_MAXE_result.append(Holdout_maxe)

        # Form new predicted output dataframe by concatenating dataframes
        TRAIN_PRED = pd.concat([TRAIN_PRED, Train_Pred], axis=1)
        HOLDOUT_PRED = pd.concat([HOLDOUT_PRED, Holdout_Pred], axis=1)
        NEW_TEST_PRED_2 = pd.concat([NEW_TEST_PRED_2, New_Test_pred_2], axis=1)

        # Training regression plot
        print('Model: {} - Random_state : {} - R2: {:.5f} | MAE: {:.5f} | MSE: {:.5f} | RMSE: {:.5f} | MAPE: {:.5f} | MEDAE: {:.5f} | MAXE: {:.5f}'.format(reg_n, i,
                                                                                            Train_r2, Train_mae, Train_mse, Train_rmse,
                                                                                            Train_mape, Train_medae, Train_maxe))

        # Holdout regression plot
        print('Model: {} - Random_state : {} - R2: {:.5f} | MAE: {:.5f} | MSE: {:.5f} | RMSE: {:.5f} | MAPE: {:.5f} | MEDAE: {:.5f} | MAXE: {:.5f}'.format(reg_n, i,
                                                                                            Holdout_r2, Holdout_mae, Holdout_mse, Holdout_rmse,
                                                                                            Holdout_mape, Holdout_medae, Holdout_maxe))

    Train_R2 = pd.DataFrame(TRAIN_R2_result, columns=[str(reg_n)])
    TRAIN_R2 = pd.concat([TRAIN_R2, Train_R2],axis=1)

    Train_MAE = pd.DataFrame(TRAIN_MAE_result, columns=[str(reg_n)])
    TRAIN_MAE = pd.concat([TRAIN_MAE, Train_MAE],axis=1)

    Train_MSE = pd.DataFrame(TRAIN_MSE_result, columns=[str(reg_n)])
    TRAIN_MSE = pd.concat([TRAIN_MSE, Train_MSE],axis=1)

    Train_RMSE = pd.DataFrame(TRAIN_RMSE_result, columns=[str(reg_n)])
    TRAIN_RMSE = pd.concat([TRAIN_RMSE, Train_RMSE],axis=1)

    Train_MAPE = pd.DataFrame(TRAIN_MAPE_result, columns=[str(reg_n)])
    TRAIN_MAPE = pd.concat([TRAIN_MAPE, Train_MAPE],axis=1)

    Train_MEDAE = pd.DataFrame(TRAIN_MEDAE_result, columns=[str(reg_n)])
    TRAIN_MEDAE = pd.concat([TRAIN_MEDAE, Train_MEDAE],axis=1)

    Train_MAXE = pd.DataFrame(TRAIN_MAXE_result, columns=[str(reg_n)])
    TRAIN_MAXE = pd.concat([TRAIN_MAXE, Train_MAXE],axis=1)


    Holdout_R2 = pd.DataFrame(HOLDOUT_R2_result, columns=[str(reg_n)])
    HOLDOUT_R2 = pd.concat([HOLDOUT_R2, Holdout_R2],axis=1)

    Holdout_MAE = pd.DataFrame(HOLDOUT_MAE_result, columns=[str(reg_n)])
    HOLDOUT_MAE = pd.concat([HOLDOUT_MAE, Holdout_MAE],axis=1)

    Holdout_MSE = pd.DataFrame(HOLDOUT_MSE_result, columns=[str(reg_n)])
    HOLDOUT_MSE = pd.concat([HOLDOUT_MSE, Holdout_MSE],axis=1)

    Holdout_RMSE = pd.DataFrame(HOLDOUT_RMSE_result, columns=[str(reg_n)])
    HOLDOUT_RMSE = pd.concat([HOLDOUT_RMSE, Holdout_RMSE],axis=1)

    Holdout_MAPE = pd.DataFrame(HOLDOUT_MAPE_result, columns=[str(reg_n)])
    HOLDOUT_MAPE = pd.concat([HOLDOUT_MAPE, Holdout_MAPE],axis=1)

    Holdout_MEDAE = pd.DataFrame(HOLDOUT_MEDAE_result, columns=[str(reg_n)])
    HOLDOUT_MEDAE = pd.concat([HOLDOUT_MEDAE, Holdout_MEDAE],axis=1)

    Holdout_MAXE = pd.DataFrame(HOLDOUT_MAXE_result, columns=[str(reg_n)])
    HOLDOUT_MAXE = pd.concat([HOLDOUT_MAXE, Holdout_MAXE],axis=1)

Thu Jul 17 09:25:00 2025
Model: skorch_model_1 - Random_state : 0 - R2: 0.87221 | MAE: 14.34153 | MSE: 565.41992 | RMSE: 23.77856 | MAPE: 23706403340288.00000 | MEDAE: 8.93599 | MAXE: 453.34500
Model: skorch_model_1 - Random_state : 0 - R2: 0.83128 | MAE: 16.35611 | MSE: 720.95074 | RMSE: 26.85053 | MAPE: 36573789814784.00000 | MEDAE: 10.13909 | MAXE: 455.63776
Thu Jul 17 09:26:39 2025
Model: skorch_model_1 - Random_state : 1 - R2: 0.87354 | MAE: 14.26063 | MSE: 559.56934 | RMSE: 23.65522 | MAPE: 22194144935936.00000 | MEDAE: 8.89867 | MAXE: 449.94119
Model: skorch_model_1 - Random_state : 1 - R2: 0.83225 | MAE: 16.28318 | MSE: 716.84247 | RMSE: 26.77391 | MAPE: 31785748529152.00000 | MEDAE: 9.93257 | MAXE: 449.22461
Thu Jul 17 09:28:13 2025
Model: skorch_model_1 - Random_state : 2 - R2: 0.87147 | MAE: 14.33131 | MSE: 568.72180 | RMSE: 23.84789 | MAPE: 20525732593664.00000 | MEDAE: 8.94913 | MAXE: 464.28516
Model: skorch_model_1 - Random_state : 2 - R2: 0.83122 | MAE: 16.34703 | MSE: 7

In [ ]:
TRAIN_R2

,skorch_model_1,skorch_model_2,skorch_model_3
0,0.872215,0.871503,0.873117
1,0.873537,0.871755,0.874027
2,0.871469,0.874282,0.872221
3,0.871864,0.872361,0.872801
4,0.873317,0.871691,0.871763
...,...,...,...
95,0.871230,0.871702,0.873153
96,0.872757,0.872740,0.870513
97,0.872766,0.872178,0.871409
98,0.873365,0.872433,0.871958


In [ ]:
TRAIN_R2.to_csv('K_VRH_TRAIN_R2.csv', index=False)
!head K_VRH_TRAIN_R2.csv

skorch_model_1,skorch_model_2,skorch_model_3
0.8722149026220415,0.871503427786078,0.873117249811427
0.8735371546682639,0.8717550588086866,0.8740265635407682
0.8714686875296007,0.874282055401373,0.8722213499786103
0.8718638986679834,0.8723614218060877,0.8728009770591312
0.8733170404095044,0.8716909066885973,0.8717632952434082
0.8718623982620227,0.8718338030349593,0.8729524245491198
0.8724558405795928,0.8721898066238956,0.8722581172527378
0.872219781609095,0.8713789315990284,0.8733246099520704
0.8739680609842363,0.8717518066724855,0.8718201569838144


In [ ]:
TRAIN_MAE

,skorch_model_1,skorch_model_2,skorch_model_3
0,14.341535,14.380056,14.219858
1,14.260630,14.259467,14.165888
2,14.331308,14.149402,14.311016
3,14.272033,14.312722,14.259294
4,14.222944,14.360164,14.243293
...,...,...,...
95,14.290555,14.291961,14.222878
96,14.262835,14.310443,14.445477
97,14.278778,14.341303,14.323246
98,14.257596,14.274837,14.311018


In [ ]:
TRAIN_MAE.to_csv('K_VRH_TRAIN_MAE.csv', index=False)
!head K_VRH_TRAIN_MAE.csv

skorch_model_1,skorch_model_2,skorch_model_3
14.341535,14.380056,14.219858
14.26063,14.259467,14.165888
14.331308,14.149402,14.311016
14.272033,14.312722,14.259294
14.222944,14.360164,14.243293
14.29791,14.376238,14.271851
14.203949,14.343456,14.275328
14.3444195,14.363863,14.216314
14.249754,14.294096,14.33741


In [ ]:
TRAIN_MSE

,skorch_model_1,skorch_model_2,skorch_model_3
0,565.419922,568.567993,561.427307
1,559.569336,567.454651,557.403748
2,568.721802,556.273315,565.391418
3,566.973083,564.771606,562.826660
4,560.543274,567.738464,567.418213
...,...,...,...
95,569.776123,567.689697,561.269897
96,563.020325,563.098328,572.948792
97,562.980896,565.581909,568.984070
98,560.330322,564.455139,566.557617


In [ ]:
TRAIN_MSE.to_csv('K_VRH_TRAIN_MSE.csv', index=False)
!head K_VRH_TRAIN_MSE.csv

skorch_model_1,skorch_model_2,skorch_model_3
565.4199,568.568,561.4273
559.56934,567.45465,557.40375
568.7218,556.2733,565.3914
566.9731,564.7716,562.82666
560.5433,567.73846,567.4182
566.9797,567.1062,562.15656
564.3538,565.531,565.22876
565.3984,569.1189,560.50977
557.6626,567.46906,567.16656


In [ ]:
TRAIN_RMSE

,skorch_model_1,skorch_model_2,skorch_model_3
0,23.778561,23.844664,23.694458
1,23.655218,23.821306,23.609400
2,23.847889,23.585447,23.777960
3,23.811197,23.764923,23.723969
4,23.675795,23.827263,23.820541
...,...,...,...
95,23.869984,23.826241,23.691135
96,23.728050,23.729692,23.936348
97,23.727219,23.781965,23.853388
98,23.671297,23.758265,23.802471


In [ ]:
TRAIN_RMSE.to_csv('K_VRH_TRAIN_RMSE.csv', index=False)
!head K_VRH_TRAIN_RMSE.csv

skorch_model_1,skorch_model_2,skorch_model_3
23.77856,23.844664,23.694458
23.655218,23.821306,23.6094
23.847889,23.585447,23.77796
23.811197,23.764923,23.723969
23.675795,23.827263,23.820541
23.811335,23.813992,23.70984
23.756132,23.780895,23.77454
23.778107,23.856213,23.675087
23.614882,23.82161,23.81526


In [ ]:
TRAIN_MAPE

,skorch_model_1,skorch_model_2,skorch_model_3
0,2.370640e+13,2.369787e+13,2.424004e+13
1,2.219414e+13,2.322111e+13,2.599571e+13
2,2.052573e+13,2.748692e+13,2.509280e+13
3,2.182621e+13,2.185964e+13,2.448850e+13
4,2.308454e+13,2.354500e+13,2.033373e+13
...,...,...,...
95,2.399569e+13,2.097713e+13,2.246252e+13
96,2.450506e+13,2.365326e+13,2.332203e+13
97,2.408630e+13,1.931151e+13,2.352028e+13
98,2.125188e+13,2.047925e+13,2.378422e+13


In [ ]:
TRAIN_MAPE.to_csv('K_VRH_TRAIN_MAPE.csv', index=False)
!head K_VRH_TRAIN_MAPE.csv

skorch_model_1,skorch_model_2,skorch_model_3
23706403000000.0,23697868000000.0,24240038000000.0
22194145000000.0,23221114000000.0,25995710000000.0
20525733000000.0,27486925000000.0,25092801000000.0
21826212000000.0,21859643000000.0,24488503000000.0
23084537000000.0,23545002000000.0,20333726000000.0
24386308000000.0,26042515000000.0,23493777000000.0
21160580000000.0,25074363000000.0,24794198000000.0
24359895000000.0,22665434000000.0,22135823000000.0
24601598000000.0,21702778000000.0,21762675000000.0


In [ ]:
TRAIN_MEDAE

,skorch_model_1,skorch_model_2,skorch_model_3
0,8.935989,8.966795,8.842937
1,8.898666,8.826532,8.717491
2,8.949135,8.680670,8.771383
3,8.817440,8.956688,8.940498
4,8.841402,8.988434,8.663620
...,...,...,...
95,8.881929,8.796430,8.819433
96,8.872588,8.966935,8.950161
97,8.926933,8.891270,8.838572
98,8.836546,8.823124,8.910370


In [ ]:
TRAIN_MEDAE.to_csv('K_VRH_TRAIN_MEDAE.csv', index=False)
!head K_VRH_TRAIN_MEDAE.csv

skorch_model_1,skorch_model_2,skorch_model_3
8.935989,8.966795,8.842937
8.898666,8.826532,8.717491
8.949135,8.68067,8.771383
8.81744,8.956688,8.940498
8.841402,8.988434,8.66362
8.852905,9.052113,8.811215
8.76902,8.997097,8.874016
8.918129,8.977394,8.782019
8.890126,8.839428,8.904623


In [ ]:
TRAIN_MAXE

,skorch_model_1,skorch_model_2,skorch_model_3
0,453.345001,447.979767,476.583008
1,449.941193,472.695679,437.261688
2,464.285156,462.398407,454.793396
3,469.650513,464.966187,459.803680
4,474.013702,474.533417,471.744629
...,...,...,...
95,471.458191,461.406067,463.467010
96,470.938141,445.134766,438.684021
97,463.000610,455.848816,476.796814
98,464.874695,469.832489,462.687134


In [ ]:
TRAIN_MAXE.to_csv('K_VRH_TRAIN_MAXE.csv', index=False)
!head K_VRH_TRAIN_MAXE.csv

skorch_model_1,skorch_model_2,skorch_model_3
453.345,447.97977,476.583
449.9412,472.69568,437.2617
464.28516,462.3984,454.7934
469.6505,464.9662,459.80368
474.0137,474.53342,471.74463
473.85803,481.73138,465.27557
484.9862,449.4044,463.15924
458.45892,465.81265,463.4325
450.61148,471.61786,471.5981


In [ ]:
HOLDOUT_R2

,skorch_model_1,skorch_model_2,skorch_model_3
0,0.831284,0.830861,0.832476
1,0.832246,0.832895,0.832575
2,0.831218,0.833800,0.831730
3,0.833803,0.832016,0.831619
4,0.833903,0.830876,0.834606
...,...,...,...
95,0.832030,0.831526,0.833410
96,0.833730,0.833391,0.830406
97,0.831143,0.831542,0.829975
98,0.833288,0.832160,0.831301


In [ ]:
HOLDOUT_R2.to_csv('K_VRH_HOLDOUT_R2.csv', index=False)
!head K_VRH_HOLDOUT_R2.csv

skorch_model_1,skorch_model_2,skorch_model_3
0.8312843198795948,0.8308607375573708,0.8324764729777316
0.832245725381522,0.8328948392076841,0.8325750664559606
0.8312176193672645,0.8338002574387915,0.8317298539902828
0.8338027379514956,0.8320161070841025,0.8316188638632654
0.8339031147847413,0.8308758729127705,0.8346063613230189
0.8334716813292793,0.8318236893129145,0.8320316231709336
0.8328690163462483,0.830410712984843,0.8295016806810366
0.8321347519687023,0.8327735077753342,0.8314948416704635
0.8311299168012496,0.8298886794324393,0.8307993571231542


In [ ]:
HOLDOUT_MAE

,skorch_model_1,skorch_model_2,skorch_model_3
0,16.356106,16.325878,16.255745
1,16.283178,16.204065,16.194715
2,16.347029,16.238480,16.323996
3,16.271257,16.312660,16.302504
4,16.163223,16.420725,16.189548
...,...,...,...
95,16.214031,16.366491,16.253408
96,16.245039,16.362127,16.382538
97,16.220373,16.379589,16.425400
98,16.252907,16.225689,16.232780


In [ ]:
HOLDOUT_MAE.to_csv('K_VRH_HOLDOUT_MAE.csv', index=False)
!head K_VRH_HOLDOUT_MAE.csv

skorch_model_1,skorch_model_2,skorch_model_3
16.356106,16.325878,16.255745
16.283178,16.204065,16.194715
16.347029,16.23848,16.323996
16.271257,16.31266,16.302504
16.163223,16.420725,16.189548
16.205622,16.427027,16.288303
16.291061,16.487284,16.487047
16.193668,16.439217,16.266733
16.412483,16.361795,16.389944


In [ ]:
HOLDOUT_MSE

,skorch_model_1,skorch_model_2,skorch_model_3
0,720.950745,722.760803,715.856506
1,716.842468,714.068787,715.435181
2,721.235657,710.199707,719.046936
3,710.189087,717.823669,719.521118
4,709.760254,722.696167,706.755066
...,...,...,...
95,717.765808,719.916260,711.865356
96,710.498047,711.948425,724.703369
97,721.553040,719.850952,726.547241
98,712.388245,717.210266,720.877380


In [ ]:
HOLDOUT_MSE.to_csv('K_VRH_HOLDOUT_MSE.csv', index=False)
!head K_VRH_HOLDOUT_MSE.csv

skorch_model_1,skorch_model_2,skorch_model_3
720.95074,722.7608,715.8565
716.84247,714.0688,715.4352
721.23566,710.1997,719.04694
710.1891,717.82367,719.5211
709.76025,722.69617,706.75507
711.60376,718.6459,717.7573
714.179,724.6839,728.56824
717.3167,714.5872,720.05115
721.6106,726.91455,723.0231


In [ ]:
HOLDOUT_RMSE

,skorch_model_1,skorch_model_2,skorch_model_3
0,26.850527,26.884211,26.755495
1,26.773914,26.722065,26.747620
2,26.855831,26.649572,26.815050
3,26.649374,26.792231,26.823891
4,26.641327,26.883009,26.584866
...,...,...,...
95,26.791151,26.831255,26.680805
96,26.655169,26.682362,26.920315
97,26.861740,26.830038,26.954540
98,26.690601,26.780781,26.849159


In [ ]:
HOLDOUT_RMSE.to_csv('K_VRH_HOLDOUT_RMSE.csv', index=False)
!head K_VRH_HOLDOUT_RMSE.csv

skorch_model_1,skorch_model_2,skorch_model_3
26.850527,26.88421,26.755495
26.773914,26.722065,26.74762
26.855831,26.649572,26.81505
26.649374,26.79223,26.82389
26.641327,26.883009,26.584866
26.675901,26.807571,26.790993
26.724129,26.919954,26.992002
26.782768,26.731764,26.833769
26.86281,26.961353,26.889088


In [ ]:
HOLDOUT_MAPE

,skorch_model_1,skorch_model_2,skorch_model_3
0,3.657379e+13,3.289711e+13,3.234597e+13
1,3.178575e+13,2.848220e+13,3.785936e+13
2,2.884684e+13,3.746056e+13,3.662214e+13
3,3.175947e+13,2.271637e+13,3.571168e+13
4,2.903798e+13,3.258958e+13,2.741366e+13
...,...,...,...
95,3.540099e+13,2.872552e+13,2.983118e+13
96,3.384153e+13,3.077300e+13,2.996406e+13
97,4.008462e+13,3.543090e+13,3.743345e+13
98,2.498066e+13,2.465109e+13,3.550787e+13


In [ ]:
HOLDOUT_MAPE.to_csv('K_VRH_HOLDOUT_MAPE.csv', index=False)
!head K_VRH_HOLDOUT_MAPE.csv

skorch_model_1,skorch_model_2,skorch_model_3
36573790000000.0,32897107000000.0,32345973000000.0
31785749000000.0,28482201000000.0,37859357000000.0
28846837000000.0,37460560000000.0,36622138000000.0
31759471000000.0,22716370000000.0,35711684000000.0
29037976000000.0,32589583000000.0,27413665000000.0
38118460000000.0,36165300000000.0,34365768000000.0
27917373000000.0,29430955000000.0,41156260000000.0
33943454000000.0,28407163000000.0,40021700000000.0
41847485000000.0,30548198000000.0,31533302000000.0


In [ ]:
HOLDOUT_MEDAE

,skorch_model_1,skorch_model_2,skorch_model_3
0,10.139088,9.888420,10.025806
1,9.932568,9.898118,9.776810
2,9.885208,10.003822,10.043781
3,10.026407,9.926121,9.749710
4,9.756256,10.145561,10.123463
...,...,...,...
95,9.907455,10.079010,9.912994
96,9.977713,10.304749,10.090164
97,10.016088,10.029465,10.089897
98,9.771782,10.028854,9.707001


In [ ]:
HOLDOUT_MEDAE.to_csv('K_VRH_HOLDOUT_MEDAE.csv', index=False)
!head K_VRH_HOLDOUT_MEDAE.csv

skorch_model_1,skorch_model_2,skorch_model_3
10.139088,9.88842,10.025806
9.932568,9.898118,9.77681
9.885208,10.003822,10.043781
10.026407,9.926121,9.74971
9.756256,10.145561,10.123463
10.1116905,10.248856,10.127335
9.966164,10.188459,10.170731
9.90641,10.244959,9.841335
9.967386,9.940819,10.379974


In [ ]:
HOLDOUT_MAXE

,skorch_model_1,skorch_model_2,skorch_model_3
0,455.637756,457.396332,458.334259
1,449.224609,457.302521,449.013245
2,447.035339,441.157837,443.744629
3,451.568390,454.449463,450.082001
4,452.932037,449.889191,455.105560
...,...,...,...
95,451.419586,451.522583,452.825592
96,445.806519,444.016296,451.684540
97,457.087585,452.201935,449.966309
98,450.442780,454.536194,453.277496


In [ ]:
HOLDOUT_MAXE.to_csv('K_VRH_HOLDOUT_MAXE.csv', index=False)
!head K_VRH_HOLDOUT_MAXE.csv

skorch_model_1,skorch_model_2,skorch_model_3
455.63776,457.39633,458.33426
449.2246,457.30252,449.01324
447.03534,441.15784,443.74463
451.5684,454.44946,450.082
452.93204,449.8892,455.10556
455.52133,448.31702,454.05804
455.5497,453.2158,455.6686
452.39288,450.62326,450.81494
444.30026,449.7728,452.17313


In [ ]:
NEW_TEST_PRED_2

,skorch_model_1_0,skorch_model_1_1,skorch_model_1_2,skorch_model_1_3,skorch_model_1_4,skorch_model_1_5,skorch_model_1_6,skorch_model_1_7,skorch_model_1_8,skorch_model_1_9,...,skorch_model_3_90,skorch_model_3_91,skorch_model_3_92,skorch_model_3_93,skorch_model_3_94,skorch_model_3_95,skorch_model_3_96,skorch_model_3_97,skorch_model_3_98,skorch_model_3_99
0,24.233536,32.572861,27.625238,27.897530,28.662012,29.251369,29.823986,29.010696,23.803280,23.043884,...,27.184361,28.768969,24.583015,25.278072,28.119629,30.058271,29.124460,26.650692,25.831900,28.140053
1,72.410133,70.059418,71.705894,69.655006,71.602631,78.912361,75.717323,67.869919,74.312141,67.741119,...,73.834427,69.028931,72.419357,70.309105,69.462776,69.043976,70.340530,73.766373,74.790085,67.813133
2,72.724770,73.982468,73.738335,74.314331,77.212669,78.801735,76.504326,72.313812,74.613716,75.758614,...,77.027817,75.517906,75.285934,76.340828,72.331779,73.023643,72.554451,76.262741,75.108604,74.394020
3,26.292654,39.248867,32.524067,36.237869,36.816223,37.950752,34.450512,35.323471,31.221498,33.302578,...,38.265572,35.818954,37.358414,32.379745,37.868149,34.268742,34.719616,33.795536,36.307476,35.531040
4,83.737091,83.919228,87.594048,88.555565,88.470757,87.802765,84.752129,84.600601,84.478951,84.746437,...,87.134338,80.523567,82.434250,85.481422,80.683998,85.531975,83.229088,85.575951,89.335541,86.933960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4738,32.715260,40.112541,25.434921,34.302311,36.677902,32.679749,36.882580,36.488052,46.999588,33.467300,...,31.625223,27.301346,28.637611,44.292854,38.167679,28.304285,26.815165,30.553623,39.919373,27.429680
4739,30.647133,41.079098,27.631800,30.523668,35.452198,32.790848,36.000355,38.783421,46.363144,31.901899,...,35.259171,28.073505,31.072950,47.445656,34.342251,28.407707,24.664949,31.992893,32.168774,28.307281
4740,31.783653,37.606945,21.342014,34.803680,33.468048,26.791714,32.326210,34.031555,41.318752,31.780983,...,28.413189,24.668070,28.798151,36.974564,34.312763,30.504591,24.627745,27.757088,37.882835,22.333113
4741,25.664410,40.838715,16.840372,28.302914,28.055561,18.959583,25.530806,19.177319,29.638704,26.640963,...,25.206953,21.073109,24.644382,23.210159,26.221075,26.893694,24.419813,25.649160,38.213650,22.402781


In [ ]:
NEW_TEST_PRED_2.to_csv('NEW_K_VRH_predictions_for_SCREENED_Li-based_Na_based_K-based_Mg-based_Ba-based_Ca-based_and_Al-based_compounds_from_MPJ_with_no_mech_prop_info.csv', index=False)
!head NEW_K_VRH_predictions_for_SCREENED_Li-based_Na_based_K-based_Mg-based_Ba-based_Ca-based_and_Al-based_compounds_from_MPJ_with_no_mech_prop_info.csv

skorch_model_1_0,skorch_model_1_1,skorch_model_1_2,skorch_model_1_3,skorch_model_1_4,skorch_model_1_5,skorch_model_1_6,skorch_model_1_7,skorch_model_1_8,skorch_model_1_9,skorch_model_1_10,skorch_model_1_11,skorch_model_1_12,skorch_model_1_13,skorch_model_1_14,skorch_model_1_15,skorch_model_1_16,skorch_model_1_17,skorch_model_1_18,skorch_model_1_19,skorch_model_1_20,skorch_model_1_21,skorch_model_1_22,skorch_model_1_23,skorch_model_1_24,skorch_model_1_25,skorch_model_1_26,skorch_model_1_27,skorch_model_1_28,skorch_model_1_29,skorch_model_1_30,skorch_model_1_31,skorch_model_1_32,skorch_model_1_33,skorch_model_1_34,skorch_model_1_35,skorch_model_1_36,skorch_model_1_37,skorch_model_1_38,skorch_model_1_39,skorch_model_1_40,skorch_model_1_41,skorch_model_1_42,skorch_model_1_43,skorch_model_1_44,skorch_model_1_45,skorch_model_1_46,skorch_model_1_47,skorch_model_1_48,skorch_model_1_49,skorch_model_1_50,skorch_model_1_51,skorch_model_1_52,skorch_model_1_53,skorch_model_1_54,skorch_model_1_55,sk

In [ ]:
TRAIN_R2.describe()

,skorch_model_1,skorch_model_2,skorch_model_3
count,100.000000,100.000000,100.000000
mean,0.872390,0.872387,0.872478
std,0.000920,0.000936,0.000895
min,0.869510,0.870060,0.870513
25%,0.871861,0.871754,0.871820
50%,0.872347,0.872258,0.872510
75%,0.872955,0.872960,0.873124
max,0.874891,0.874801,0.874724


In [ ]:
HOLDOUT_R2.describe()

,skorch_model_1,skorch_model_2,skorch_model_3
count,100.000000,100.000000,100.000000
mean,0.832142,0.832167,0.832016
std,0.001233,0.001223,0.001316
min,0.829462,0.829677,0.828775
25%,0.831387,0.831330,0.831160
50%,0.832141,0.832122,0.832036
75%,0.832814,0.832788,0.832810
max,0.836179,0.835304,0.835067


In [ ]:
TRAIN_MAE.describe()

,skorch_model_1,skorch_model_2,skorch_model_3
count,100.000000,100.000000,100.000000
mean,14.284659,14.285452,14.282327
std,0.069581,0.068962,0.061629
min,14.070735,14.125953,14.122789
25%,14.249561,14.231201,14.240959
50%,14.288034,14.291735,14.279894
75%,14.329940,14.342408,14.325437
max,14.464862,14.416105,14.445477


In [ ]:
HOLDOUT_MAE.describe()

,skorch_model_1,skorch_model_2,skorch_model_3
count,100.000000,100.000000,100.000000
mean,16.307076,16.316460,16.326664
std,0.076919,0.083296,0.084363
min,16.163223,16.112627,16.101152
25%,16.252818,16.256410,16.261632
50%,16.293557,16.321672,16.331099
75%,16.352210,16.381075,16.389575
max,16.543484,16.487284,16.521149


In [ ]:
TRAIN_MSE.describe()

,skorch_model_1,skorch_model_2,skorch_model_3
count,100.000000,100.000000,100.000000
mean,564.643005,564.658264,564.255066
std,4.070825,4.141003,3.959265
min,553.580261,553.974915,554.318481
25%,562.147385,562.121933,561.395874
50%,564.836090,565.228638,564.114014
75%,566.987335,567.458252,567.168198
max,577.388428,574.953552,572.948792


In [ ]:
HOLDOUT_MSE.describe()

,skorch_model_1,skorch_model_2,skorch_model_3
count,100.000000,100.000000,100.000000
mean,717.286621,717.179688,717.825317
std,5.269783,5.227098,5.623761
min,700.032898,703.772949,704.788574
25%,714.414490,714.523529,714.430389
50%,717.290131,717.372833,717.739777
75%,720.510712,720.753510,721.481720
max,728.737183,727.820251,731.671509


In [ ]:
TRAIN_RMSE.describe()

,skorch_model_1,skorch_model_2,skorch_model_3
count,100.000000,100.000000,100.000000
mean,23.762068,23.762383,23.753904
std,0.085637,0.087209,0.083330
min,23.528286,23.536673,23.543968
25%,23.709648,23.709111,23.693795
50%,23.766280,23.774538,23.751084
75%,23.811496,23.821382,23.815294
max,24.028908,23.978189,23.936348


In [ ]:
HOLDOUT_RMSE.describe()

,skorch_model_1,skorch_model_2,skorch_model_3
count,100.000000,100.000000,100.000000
mean,26.782032,26.780037,26.792061
std,0.098422,0.097634,0.104956
min,26.458136,26.528719,26.547853
25%,26.728534,26.730573,26.728830
50%,26.782272,26.783816,26.790666
75%,26.842331,26.846853,26.860412
max,26.995132,26.978144,27.049427


In [ ]:
TRAIN_MAPE.describe()

,skorch_model_1,skorch_model_2,skorch_model_3
count,1.000000e+02,1.000000e+02,1.000000e+02
mean,2.399494e+13,2.403936e+13,2.415959e+13
std,2.117699e+12,2.187340e+12,1.927802e+12
min,1.870677e+13,1.931151e+13,1.923529e+13
25%,2.264910e+13,2.266390e+13,2.288152e+13
50%,2.407357e+13,2.400238e+13,2.411745e+13
75%,2.512471e+13,2.537095e+13,2.547375e+13
max,3.038698e+13,2.916326e+13,2.822341e+13


In [ ]:
HOLDOUT_MAPE.describe()

,skorch_model_1,skorch_model_2,skorch_model_3
count,1.000000e+02,1.000000e+02,1.000000e+02
mean,3.330989e+13,3.252196e+13,3.275301e+13
std,4.197398e+12,4.500053e+12,4.229646e+12
min,2.449464e+13,2.271637e+13,2.350829e+13
25%,3.048472e+13,2.960116e+13,2.980855e+13
50%,3.329068e+13,3.245100e+13,3.278113e+13
75%,3.624680e+13,3.492617e+13,3.543306e+13
max,4.610116e+13,4.646111e+13,4.373637e+13


In [ ]:
TRAIN_MEDAE.describe()

,skorch_model_1,skorch_model_2,skorch_model_3
count,100.000000,100.000000,100.000000
mean,8.856050,8.864944,8.852848
std,0.088951,0.099493,0.089030
min,8.640593,8.648897,8.614017
25%,8.787909,8.801633,8.795113
50%,8.861313,8.859027,8.842699
75%,8.901776,8.947496,8.926925
max,9.156519,9.052113,9.026360


In [ ]:
HOLDOUT_MEDAE.describe()

,skorch_model_1,skorch_model_2,skorch_model_3
count,100.000000,100.000000,100.000000
mean,10.028326,10.050693,10.039580
std,0.144540,0.149392,0.154321
min,9.738708,9.642662,9.707001
25%,9.924868,9.939878,9.948328
50%,10.019110,10.035158,10.051474
75%,10.128326,10.146441,10.133755
max,10.555481,10.412140,10.379974


In [ ]:
TRAIN_MAXE.describe()

,skorch_model_1,skorch_model_2,skorch_model_3
count,100.000000,100.000000,100.000000
mean,467.192932,466.026764,464.325592
std,10.509354,10.355483,11.448888
min,438.923157,439.141052,437.261688
25%,462.404015,460.225792,457.476509
50%,468.860245,466.966232,465.512695
75%,474.232018,473.383820,472.370941
max,487.539246,487.157166,488.565033


In [ ]:
HOLDOUT_MAXE.describe()

,skorch_model_1,skorch_model_2,skorch_model_3
count,100.000000,100.000000,100.000000
mean,450.983948,451.224060,451.593628
std,3.832471,3.371546,3.644383
min,441.138336,441.157837,440.013733
25%,448.625046,449.301567,449.034576
50%,450.454086,451.601166,451.711304
75%,454.309990,453.367485,453.676025
max,458.962250,458.076416,461.748962
